In [1]:
import argparse
import os
import sys
import time
import tensorflow as tf
import tushare as ts
import pandas as pd
import numpy as np
import stock
import data_collecter_sohu as sohu_dc
from six.moves import xrange  # pylint: disable=redefined-builtin
import argparse

FLAGS=argparse.Namespace()
FLAGS.learning_rate=0.03
FLAGS.max_steps=300000
FLAGS.hidden1=70
FLAGS.hidden2=20
FLAGS.batch_size=300
FLAGS.input_data_dir=os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),'tensorflow/stock/input_data')
FLAGS.log_dir=os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),'tensorflow/stock/logs/fully_connected_feed')
FLAGS.fake_data=False

FLAGS

Namespace(batch_size=300, fake_data=False, hidden1=70, hidden2=20, input_data_dir='/tmp/tensorflow/stock/input_data', learning_rate=0.03, log_dir='/tmp/tensorflow/stock/logs/fully_connected_feed', max_steps=300000)

In [3]:
def placeholder_inputs(batch_size):
  """Generate placeholder variables to represent the input tensors.

  These placeholders are used as inputs by the rest of the model building
  code and will be fed from the downloaded data in the .run() loop, below.

  Args:
    batch_size: The batch size will be baked into both placeholders.

  Returns:
    features_placeholder: Features placeholder.
    labels_placeholder: Labels placeholder.
  """
  # Note that the shapes of the placeholders match the shapes of the full
  # image and label tensors, except the first dimension is now batch_size
  # rather than the full size of the train or test data sets.
  print('features placeholder shape %d %d'%(batch_size, stock.NUM_FEATURES))
  features_placeholder = tf.placeholder(tf.float32, shape=(None,
                                                         stock.NUM_FEATURES))
  labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
  return features_placeholder, labels_placeholder

def fill_feed_dict(features_feed, labels_feed, features_pl, labels_pl):
  """Fills the feed_dict for training the given step.

  A feed_dict takes the form of:
  feed_dict = {
      <placeholder>: <tensor of values to be passed for placeholder>,
      ....
  }

  Args:
    data_set: The DataSet contains features and labels
    features_pl: The features placeholder, from placeholder_inputs().
    labels_pl: The labels placeholder, from placeholder_inputs().

  Returns:
    feed_dict: The feed dictionary mapping from placeholders to values.
  """
  feed_dict = {
      features_pl: features_feed,
      labels_pl: labels_feed,
  }
  return feed_dict

def do_eval(sess,
            eval_correct,
            features_placeholder,
            labels_placeholder,
            dataset):
  """Runs one evaluation against the full epoch of data.

  Args:
    sess: The session in which the model has been trained.
    eval_correct: The Tensor that returns the number of correct predictions.
    features_placeholder: The features placeholder.
    labels_placeholder: The labels placeholder.
    data_set: a dict contains features and labels
  """
#  eval_data = tf.data.Dataset.from_tensor_slices((np.array(dataset['features']),np.array(dataset['lables'])))
#  eval_data = eval_data.repeat()
#  eval_data = eval_data.batch(FLAGS.batch_size)
  # And run one epoch of eval.
  true_count = 0  # Counts the number of correct predictions.
  steps_per_epoch = 10
  num_examples = steps_per_epoch * FLAGS.batch_size
  iterator = dataset.make_one_shot_iterator()
  next_element = iterator.get_next()
  for step in xrange(steps_per_epoch):
      x,y = sess.run(next_element)
      feed_dict = fill_feed_dict(x,y,
                               features_placeholder,
                               labels_placeholder)
      true_count += sess.run(eval_correct, feed_dict=feed_dict)
  precision = float(true_count) / num_examples
  print('Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
        (num_examples, true_count, precision))


In [4]:
FLAGS.log_dir

'/tmp/tensorflow/stock/logs/fully_connected_feed'

In [5]:
if tf.gfile.Exists(FLAGS.log_dir):
    tf.gfile.DeleteRecursively(FLAGS.log_dir)
tf.gfile.MakeDirs(FLAGS.log_dir)

In [6]:
"""Train STOCk for a number of steps."""
  #df_002024=ts.get_hist_data('002024',start='2014-01-01')
if True:
  df_002024=sohu_dc.get_hist_data('002024','20080101','20171204')
  df_002024_norm=(df_002024-df_002024.min())/(df_002024.max()-df_002024.min())

  market_data={'train':{},'test':{}}
  market_data['train']['features']=[]
  market_data['train']['lables']=[]
  market_data['test']['features']=[]
  market_data['test']['lables']=[]
  features=[]
  lables=[]

  for index in range(df_002024.index.size):
      slice=df_002024_norm[df_002024.index.size-6-index:df_002024.index.size-index]
      if(slice.index.size<6):
          break
      features.append(slice[1:6].values.flatten())
      if (slice.close[0]-slice.close[1])/slice.close[1]>0.03:
          lables.append(0)
      else:
          lables.append(1) 

  test_indexes=np.random.randint(len(features),size=len(features)//10, dtype='int')
  train_indexes=[x for x in range(len(features)) if x not in test_indexes]
  print('total %d samples, %d for training, %d for test '%(len(features), len(train_indexes), len(test_indexes)))
  
  market_data['train']['features'] = [features[i] for i in train_indexes]
  market_data['train']['lables'] = [lables[i] for i in train_indexes]
  market_data['test']['features'] = [features[i] for i in test_indexes]
  market_data['test']['lables'] = [lables[i] for i in test_indexes]

  # Tell TensorFlow that the model will be built into the default Graph.
  with tf.Graph().as_default():
    dataset_train = tf.data.Dataset.from_tensor_slices((np.array(market_data['train']['features']),np.array(market_data['train']['lables'])))
    dataset_test = tf.data.Dataset.from_tensor_slices((np.array(market_data['test']['features']),np.array(market_data['test']['lables'])))

    print('batch size %d'%(FLAGS.batch_size))
    dataset_train = dataset_train.repeat().batch(FLAGS.batch_size)
    dataset_test = dataset_test.repeat().batch(FLAGS.batch_size)
    iterator = tf.data.Iterator.from_structure(dataset_train.output_types,
                                           dataset_train.output_shapes)
    next_element = iterator.get_next()
    training_init_op = iterator.make_initializer(dataset_train)
    #validation_init_op = iterator.make_initializer(dataset_test)
    # Generate placeholders for the images and labels.
    features_placeholder, labels_placeholder = placeholder_inputs(
        FLAGS.batch_size)

    # Build a Graph that computes predictions from the inference model.
    logits = stock.inference(features_placeholder,
                             FLAGS.hidden1,
                             FLAGS.hidden2)

    # Add to the Graph the Ops for loss calculation.
    loss = stock.loss(logits, labels_placeholder)

    # Add to the Graph the Ops that calculate and apply gradients.
    train_op = stock.training(loss, FLAGS.learning_rate)

    # Add the Op to compare the logits to the labels during evaluation.
    eval_correct = stock.evaluation(logits, labels_placeholder)

    # Build the summary Tensor based on the TF collection of Summaries.
    summary = tf.summary.merge_all()

    # Add the variable initializer Op.
    init = tf.global_variables_initializer()

    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver()

    # Create a session for running Ops on the Graph.
    sess = tf.Session()

    # Instantiate a SummaryWriter to output summaries and the Graph.
    summary_writer = tf.summary.FileWriter(FLAGS.log_dir, sess.graph)

    # And then after everything is built:

    # Run the Op to initialize the variables.
    sess.run(init)
    sess.run(training_init_op)
    # Start the training loop.
    for step in xrange(FLAGS.max_steps):
        start_time = time.time()
        
        x,y=sess.run(next_element)
        feed_dict = fill_feed_dict(x,y,
                                 features_placeholder,
                                 labels_placeholder)
                # Run one step of the model.  The return values are the activations
                # from the `train_op` (which is discarded) and the `loss` Op.  To
                # inspect the values of your Ops or variables, you may include them
                # in the list passed to sess.run() and the value tensors will be
                # returned in the tuple from the call.
        _, loss_value = sess.run([train_op, loss],
                               feed_dict=feed_dict)
        
        duration = time.time() - start_time
        
        if step % 1000 == 0:       
                # Write the summaries and print an overview fairly often.
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
                # Update the events file.
                summary_str = sess.run(summary, feed_dict=feed_dict)
                summary_writer.add_summary(summary_str, step)
                summary_writer.flush()
                # reinit the iterator
                sess.run(training_init_op)

        # Save a checkpoint and evaluate the model periodically.
        if (step + 1) % 20000 == 0 or (step + 1) == FLAGS.max_steps:
            checkpoint_file = os.path.join(FLAGS.log_dir, 'model.ckpt')
            saver.save(sess, checkpoint_file, global_step=step)
        # Evaluate against the training set.
            print('Training Data Eval:')
            do_eval(sess,
                eval_correct,
                features_placeholder,
                labels_placeholder,
                dataset_train)
        # Evaluate against the validation set.
            print('Test Data Eval:')
            do_eval(sess,
                eval_correct,
                features_placeholder,
                labels_placeholder,
                dataset_test)

/home/tr/bin/anaconda2/envs/python3/lib/python3.5/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars


total 2379 samples, 2151 for training, 237 for test 
batch size 300
features placeholder shape 300 85
Step 0: loss = 0.62 (0.009 sec)
Step 1000: loss = 0.48 (0.001 sec)
Step 2000: loss = 0.48 (0.002 sec)
Step 3000: loss = 0.48 (0.001 sec)
Step 4000: loss = 0.48 (0.001 sec)
Step 5000: loss = 0.48 (0.001 sec)
Step 6000: loss = 0.48 (0.001 sec)
Step 7000: loss = 0.48 (0.002 sec)
Step 8000: loss = 0.48 (0.001 sec)
Step 9000: loss = 0.47 (0.001 sec)
Step 10000: loss = 0.47 (0.001 sec)
Step 11000: loss = 0.47 (0.002 sec)
Step 12000: loss = 0.47 (0.001 sec)
Step 13000: loss = 0.47 (0.001 sec)
Step 14000: loss = 0.47 (0.001 sec)
Step 15000: loss = 0.47 (0.001 sec)
Step 16000: loss = 0.47 (0.002 sec)
Step 17000: loss = 0.47 (0.001 sec)
Step 18000: loss = 0.47 (0.001 sec)
Step 19000: loss = 0.47 (0.001 sec)
Training Data Eval:
Num examples: 3000  Num correct: 2372  Precision @ 1: 0.7907
Test Data Eval:
Num examples: 3000  Num correct: 2311  Precision @ 1: 0.7703
Step 20000: loss = 0.47 (0.001 se

Step 184000: loss = 0.19 (0.001 sec)
Step 185000: loss = 0.19 (0.001 sec)
Step 186000: loss = 0.19 (0.001 sec)
Step 187000: loss = 0.19 (0.001 sec)
Step 188000: loss = 0.18 (0.001 sec)
Step 189000: loss = 0.18 (0.001 sec)
Step 190000: loss = 0.18 (0.001 sec)
Step 191000: loss = 0.18 (0.001 sec)
Step 192000: loss = 0.17 (0.001 sec)
Step 193000: loss = 0.17 (0.001 sec)
Step 194000: loss = 0.17 (0.001 sec)
Step 195000: loss = 0.17 (0.001 sec)
Step 196000: loss = 0.17 (0.001 sec)
Step 197000: loss = 0.16 (0.001 sec)
Step 198000: loss = 0.16 (0.001 sec)
Step 199000: loss = 0.16 (0.001 sec)
Training Data Eval:
Num examples: 3000  Num correct: 2947  Precision @ 1: 0.9823
Test Data Eval:
Num examples: 3000  Num correct: 2029  Precision @ 1: 0.6763
Step 200000: loss = 0.16 (0.002 sec)
Step 201000: loss = 0.15 (0.001 sec)
Step 202000: loss = 0.15 (0.001 sec)
Step 203000: loss = 0.15 (0.001 sec)
Step 204000: loss = 0.15 (0.001 sec)
Step 205000: loss = 0.14 (0.001 sec)
Step 206000: loss = 0.14 (0.

In [9]:
s=sess.run(logits,{features_placeholder:features})

In [8]:
date=[]
for index in range(df_002024.index.size):
    slice=df_002024[df_002024.index.size-6-index:df_002024.index.size-index]
    if(slice.index.size<6):
        break
    date.append(slice.index[0])

print(len(date),s.size,type(s))
                    

2379 4758 <class 'numpy.ndarray'>


In [10]:
for i in range(s.shape[0]):
    y = np.exp(s[i])
    y1=y[0]/y.sum()
    y2=y[1]/y.sum()
    if(y1>y2):
        print(date[i],y1,y2)

20080123 0.998933 0.00106687
20080131 0.999878 0.000122036
20080204 0.998765 0.00123489
20080218 1.0 3.69176e-07
20080225 0.959243 0.0407574
20080303 0.99999 9.78035e-06
20080321 0.999997 3.02802e-06
20080403 0.993805 0.00619504
20080407 1.0 3.32222e-07
20080423 0.99529 0.00470984
20080424 0.998585 0.00141527
20080430 0.999196 0.000803701
20080508 0.999947 5.29751e-05
20080513 0.995297 0.00470312
20080528 0.999881 0.000119235
20080624 0.999073 0.000927076
20080625 0.998849 0.00115113
20080707 0.994837 0.00516258
20080708 0.996203 0.00379651
20080710 1.0 5.79174e-10
20080724 0.995444 0.00455624
20080814 1.0 2.09984e-08
20080819 0.999702 0.000297667
20080820 0.998145 0.00185457
20080903 0.999995 4.94619e-06
20080912 0.999945 5.55586e-05
20080919 0.999563 0.000436912
20080922 0.998279 0.00172111
20080925 0.998002 0.00199803
20081020 0.999818 0.000182226
20081021 1.0 5.97128e-09
20081028 0.997291 0.00270879
20081030 0.999913 8.73552e-05
20081105 0.997481 0.0025195
20081110 0.999979 2.1266e

In [12]:
test=sohu_dc.get_hist_data('002024','20171124','20171208')
test_norm=(test-test.min())/(test.max()-test.min())

In [13]:
test_features=[]
test_date=[]

for index in range(df_002024.index.size):
    slice=test_norm[test_norm.index.size-6-index:test_norm.index.size-index]
    if(slice.index.size<6):
        break
    test_features.append(slice[1:6].values.flatten())
    test_date.append(slice.index[0])
test_date

['20171201', '20171204', '20171205', '20171206', '20171207', '20171208']

In [14]:
s=sess.run(logits,{features_placeholder:test_features})
for i in range(s.shape[0]):
    y = np.exp(s[i])
    y1=y[0]/y.sum()
    y2=y[1]/y.sum()
    if(y1>y2):
        print(test_date[i],y1,y2)

20171204 0.977891 0.022109


In [ ]:
s=sess.run(logits,{features_placeholder:test_features})